# kPCA
_author: Gus Ostow_

-----
Before covering kernel methods, I will review the standard and dual algorithms for PCA, as computed by Singular Value Decomposition.

### 1. Vanilla PCA

Consider an m x n centered data matrix, $X$. An SVD computes the following factorization:

$$X = U \Sigma V^T$$

- $U$ is an m x m matrix, where the columns that are eigen-vectors of $XX^T$
- $\Sigma$ is an m x n diagonal matrix, where the diagonal entries are the $r$ non-zero singular values. Note: $r$ is the rank of $X$
- $V$ is an n x n matrix, where the columns are the eigen-vectors of $X^TX$ (the covariance matrix). These columns are the principle components of $X$.

Here is how to compute common PCA operations in terms of the SVD decomposition:

**Transform training set**

$$Y = XV_p$$

$V_p$ is the n x p truncated $V$ matrix. $p$ is the number of principle components to keep.

** Reconstruct training set**

$$\hat{X} = YV_p^T$$
$$ = XV_pV_p^T$$

Notice that $V_pV_p^T \neq I$ because they are truncated to $p$ columns, even though $VV^T = I$ by ortho-normality.

### 2. Dual PCA

Sometimes it is infeasible to directly compute the right singular vectors of $X^TX$ during a standard the standard SVD. Usually this is when there are far more columns than rows ($n >> m$), or in the case of kPCA. Luckily it is possible to solve for $V$ algebraically based on the eigen-vectors of $XX^T$, which might be easier, or only possible to solve for.

Start by computing the eigen-decomposition of $XX^T$ to construct $U$ and $\Sigma$. The diagonal entries of $\Sigma$ are the roots of the eigen-values of $XX^T$.

So now we know $X$, we know $U$ and we know $\Sigma$. The rest is just matrix algebra to solve for $V$ when you need it.

**Transform training set**

$$Y = XV_p$$

and by SVD, we know that $XV = U\Sigma$. So
$$Y = U_p\Sigma_p$$

** Reconstruct training set**

From the standard algorithm for PCA we know that

$$\hat{X} = YV_p^T$$

but we don't want to compute $V^T$ directly so we need to solve $XV = U\Sigma$ for it. We get

$$ V_p^T = \Sigma_p^{-1}U_p^TX$$

Plugging everything in, we have the full reconstruction of $X$ from a p-dimensional representation.

$$\hat{X} = U_p\Sigma_p\Sigma_p^{-1}U_p^TX$$

$$ = U_pU_p^TX$$


### 3. Kernel PCA

An assumption of standard PCA is that the structure of the data is best represented by a linear subspace. When linearity is ill-assumed, kPCA captures dynamics that lay on a sub-_**manifold**_, a potentially curved surface. The strategy: unfold the manifold into a higher dimensional space, where PCA might work better.

_**Definition:**_ 
$\Phi(x)$ is a function that transforms an $n$ dimensional vector into a $d$ dimensional vector, where $d >> n$, if not infinite-dimensional. 

_**Definition:**_ 
$K(x, y) = \langle\Phi(x), \Phi(y)\rangle$

_**Example:**_ 
Consider $X$ a 2 x 1 vector, then let $$\Phi(X) = \begin{bmatrix} x_1^2 \\ x_2^2 \\ \sqrt{2} x_1x_2  \end{bmatrix}$$

If you wanted to calculate the inner product of X and Y in this 3-dimensional space $\langle\Phi(X), \Phi(Y)\rangle$, you could figure out the dot product by hand, or you could use the kernel

$$K(X, Y) = (\langle X, Y\rangle)^2$$

Test it out yourself. They are equivalent. Anytime you want to compute a dot product in a higher-dimensional space you can use the kernel instead of actually expanding vectors into that space. THAT is the kernel trick... that's the swindle. You get to bypass the high dimensional spaces all together. They are only *implicit*.

If the dot product is a measure of vector similarity, you can think of kernels as a more-expressive, non-linear similarity measures than what is typically just an unscaled cosine similarity. We use these new similarity measures to construct a richer covariance matrix via the dual PCA algorithm.

Anywhere you see the dot product of data samples, you replace it with action by the kernel function. So now on to the algorithm.

Just as dual PCA starts with the eigen-decomposition of $XX^T$, kPCA decomposes $\Phi(X)\Phi(X^T)$, which we do not have to explicitely execute. Instead, leveraging the kernel trick, generate the columns of U and the diagonal elements of $\Sigma$ as the eigen-vectors of $K(X, X^T)$.

**Transform training set**

We transform the training set exactly the same as in dual PCA, using the "kernalized" $U$ matrix.

$$Y = U_p\Sigma_p$$

** Reconstruct training set**

You cannot reconstruct data using the kernel trick. And here's why. From dual PCA we have

$$\hat{X} = U_pU_p^TX$$

We might not necessarily be able to compute $\Phi(X)$ on it's own needed in the above. We do not have the inner product to convert using the kernel. Depending on the kernel, some reconstructions might not even exist.

### Sources

Ali Ghodsi, University of Waterloo - https://www.youtube.com/watch?v=jeOEXCFK30M&t=836s